This file is part of BrainMolecularAtlas.

This file make figure 10 (metabolites)

Copyright (c) 2021 Blue Brain Project/EPFL 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [1]:
import numpy as np
import pandas as pd
import re

import xlrd
import xlsxwriter

import altair as alt
import altair_catplot as altcat


pd.set_option('display.max_columns',100)

In [2]:
df = pd.ExcelFile('../data/4_tableS4.xlsx')
df = df.parse('Sheet1')
df = df.drop(columns='Unnamed: 0')
df.head()

,Methods,Organism,age,cellType,cid_pubchem,comments,level_units,level_units_ini,level_units_iniUniformNames,level_value_uM,locLevelOfDetails,location,methods details,molecule_name,organism/cell culture,reference,subcellularLoc,tissue
0,mass spectrometry,mouse,8-week-old,NaN,7017974,cerebra mouse 1 known,uM,nmol/g,umol/g,0.002649,tissue,cerebra mouse 1 known,ce-tofms,(2s)-2-(tert-butoxycarbonylamino)-4-oxo-4-(9h-...,mouse,"Sugimoto, 2012, Nucleic Acids Research, MMMDB:...",whole cell,brain
1,mass spectrometry,mouse,8-week-old,NaN,7017974,cerebra mouse 2 known,uM,nmol/g,umol/g,0.002752,tissue,cerebra mouse 2 known,ce-tofms,(2s)-2-(tert-butoxycarbonylamino)-4-oxo-4-(9h-...,mouse,"Sugimoto, 2012, Nucleic Acids Research, MMMDB:...",whole cell,brain
2,mass spectrometry,mouse,8-week-old,NaN,7017974,cerebella mouse 2 known,uM,nmol/g,umol/g,0.005905,tissue,cerebella mouse 2 known,ce-tofms,(2s)-2-(tert-butoxycarbonylamino)-4-oxo-4-(9h-...,mouse,"Sugimoto, 2012, Nucleic Acids Research, MMMDB:...",whole cell,brain
3,mass spectrometry,mouse,8-week-old,NaN,7017974,cerebella mouse 1 known,uM,nmol/g,umol/g,0.007253,tissue,cerebella mouse 1 known,ce-tofms,(2s)-2-(tert-butoxycarbonylamino)-4-oxo-4-(9h-...,mouse,"Sugimoto, 2012, Nucleic Acids Research, MMMDB:...",whole cell,brain
4,mass spectrometry,mouse,8-week-old,NaN,440129,cerebra mouse 1 known,uM,nmol/g,umol/g,0.001362,tissue,cerebra mouse 1 known,ce-tofms,(2s)-2-hydroxy-3-(1h-imidazol-5-yl)propanoic acid,mouse,"Sugimoto, 2012, Nucleic Acids Research, MMMDB:...",whole cell,brain


In [3]:
df = df.rename(columns={'methods_brief':'Methods'})

In [4]:
df.columns

Index(['Methods', 'Organism', 'age', 'cellType', 'cid_pubchem', 'comments',
       'level_units', 'level_units_ini', 'level_units_iniUniformNames',
       'level_value_uM', 'locLevelOfDetails', 'location', 'methods details',
       'molecule_name', 'organism/cell culture', 'reference', 'subcellularLoc',
       'tissue'],
      dtype='object')

In [5]:
df['log_level_value'] = np.log(df['level_value_uM'].astype(np.float64))

mrsmol = df['molecule_name'][df['Methods']=='mrs'].unique()
msmol = df['molecule_name'][df['Methods']=='mass spectrometry '].unique()
msmolmrsmol = list(set(msmol).intersection(mrsmol))

df['Organism'] = df['organism/cell culture'].copy()

df['Organism'][df['organism/cell culture'].isin(['human hela cells','human hct116 colon cancer cells ',
       'human glioblastoma ln229 cells in high glucose conditions',
       'human glioblastoma ln229 cells in low glucose conditions'])] = 'human'

In [8]:
chart = alt.Chart(df[(df['molecule_name'].isin(msmolmrsmol))&(df['Methods']!='database')&(df['Organism'].isin(['rat','mouse','human','human hela cells','review', 'cultured astrocytes']))].sort_values(by=['Organism']),
         width=500,
         height=500
         ).mark_point().configure_axis(labelLimit=1000).configure_legend(labelLimit=1000).encode(
    x=alt.X('log_level_value:Q', title = 'log(Concentration, uM)'),
    y=alt.Y('molecule_name:N', title=None),
    color=alt.Color('Organism:N',legend=alt.Legend(title="Organism")),
    shape=alt.Shape('Methods',legend=alt.Legend(title="Methods"))
).configure_axis(labelFontSize=14,titleFontSize=14,tickSize=12,labelLimit=1000).configure_legend(labelFontSize=14,titleFontSize=14,labelLimit=1000)

#chart.save('fig10A.html')

In [9]:
# mouse ms mrs

chart=alt.Chart(df[(df['molecule_name'].isin(msmolmrsmol))&(df['Methods'].isin(['mrs']))],
         width=500,
         height=500
         ).mark_point().configure_axis(labelLimit=1000).configure_legend(labelLimit=1000).encode(
    x=alt.X('log_level_value:Q', title = 'log(Concentration, uM)'),
    y=alt.Y('molecule_name:N', title=None),
    color=alt.Color('Organism:N',legend=alt.Legend(title="Organism")),
    #shape=alt.Shape('Organism',legend=alt.Legend(title="Organism"))
).configure_axis(labelFontSize=14,titleFontSize=14,tickSize=12,labelLimit=1000).configure_legend(labelFontSize=14,titleFontSize=14,labelLimit=1000)


#chart.save('fig10B_left.html')


In [10]:
# mouse ms mrs

chart=alt.Chart(df[(df['molecule_name'].isin(msmolmrsmol))&(df['Methods'].isin(['mass spectrometry ']))],
         width=500,
         height=500
         ).mark_point().configure_axis(labelLimit=1000).configure_legend(labelLimit=1000).encode(
    x=alt.X('log_level_value:Q', title = 'log(Concentration, uM)'),
    y=alt.Y('molecule_name:N', title=None),
    color=alt.Color('Organism:N',legend=alt.Legend(title="Organism")),
    #shape=alt.Shape('Organism',legend=alt.Legend(title="Organism"))
).configure_axis(labelFontSize=14,titleFontSize=14,tickSize=12,labelLimit=1000).configure_legend(labelFontSize=14,titleFontSize=14,labelLimit=1000)


#chart.save('fig10B_right.html')
